# О подмене функции `input` в Jupyter Notebook

Проведем эксперимент с подменой встроенной функции  
(о возможных мотивах такой замены см. комментарии к [этому твиту](https://twitter.com/vitalizzare/status/1342463345690087424), а также [размышления в конце этого ноутбука](#use_case))

In [1]:
import builtins
builtins.input = lambda prompt, _input = input: print('Hello world!') or _input(prompt)
input('Say hello: ')

Hello world!
Say hello: Hello Jupyter!


'Hello Jupyter!'

Посмотрим, как теперь ведет себя встроенная функция `input`

In [2]:
input('Say it again: ')

Say it again: Why there's no 'hello world' now?


"Why there's no 'hello world' now?"

Замена функции сработала в пределах одной ячейки. Почему?  
Проверим, изменяется ли объект, на который указывет идентификатор `input`, при переходе из одной ячейки в другую.

In [3]:
first_cell = id(input)

In [4]:
second_cell = id(input)

In [5]:
first_cell, second_cell, first_cell == second_cell

(140070900000768, 140070900011520, False)

Да, он меняется. В двух разных ячейках мы видим два разных объекта `input`  
Проверим, касается ли это других встроенных функций, например `len`

In [6]:
first_cell = id(len)

In [7]:
second_cell = id(len)

In [8]:
first_cell, second_cell, first_cell == second_cell

(140071000992080, 140071000992080, True)

`len` при переходе из одной ячейки в другую - это один и тот же объект.  
То же самое касается `print`, `hash`, `id`, `locals`, `globals`, `eval`, ...  
Что не так с `input`?

In [9]:
input.__name__

'raw_input'

In [10]:
input.__module__

'ipykernel.kernelbase'

Эта функция не из модуля `builtins`. Попробуем достать код.

In [11]:
import inspect
print(inspect.getsource(input))

    def raw_input(self, prompt=''):
        """Forward raw_input to frontends

        Raises
        ------
        StdinNotImplentedError if active frontend doesn't support stdin.
        """
        if not self._allow_stdin:
            raise StdinNotImplementedError(
                "raw_input was called, but this frontend does not support input requests."
            )
        return self._input_request(str(prompt),
            self._parent_ident,
            self._parent_header,
            password=False,
        )



Как видим, эта функция является методом какого-то класса. Более того, она представляет из себя обертку вокруг метода `self._input_request`, который, видимо, и является реальным `input`. Посмотрим, в каком классе лежит наша функция и в каком модуле находится этот класс.

In [12]:
input.__self__.__class__.__name__

'IPythonKernel'

In [13]:
input.__self__.__class__.__module__

'ipykernel.ipkernel'

Импортируем этот класс и попробуем подменить его метод `_input_request`

In [14]:
from ipykernel.ipkernel import IPythonKernel

ipython_input = IPythonKernel._input_request

def my_input(*args, **kwargs):
    print('Hello world!')
    return ipython_input(*args, **kwargs)

IPythonKernel._input_request = my_input

Проверим, что получилось.

In [15]:
input()

Hello world!
Hello Jupyter!


'Hello Jupyter!'

In [16]:
input('Say hello again: ')

Hello world!
Say hello again: Very good! We have 'hello world' now!


"Very good! We have 'hello world' now!"

In [17]:
input("Let's do it one more time: ")

Hello world!
Let's do it one more time: Yes, this is what we were looking for.


'Yes, this is what we were looking for.'

## Ищем аутентичный `input` из модуля `builtins`
Мы нашли то, что и нужно було подменять на самом деле. А где же метод `input` из модуля `builtins`? При запуске нам по умолчанию предосталяется функции `get_ipython`

In [18]:
help(get_ipython)

Help on method get_ipython in module IPython.core.interactiveshell:

get_ipython() method of ipykernel.zmqshell.ZMQInteractiveShell instance
    Return the currently running IPython instance.



Эта функция предоставляет доступ к работающему ядру. Посмотрим, нет ли внутри пространства его имен чего-то содержащего в своем названии 'input'

In [19]:
instance = get_ipython()

In [20]:
for item in dir(instance):
    if 'input' in item:
        print(item)

_last_input_line
auto_rewrite_input
extract_input_lines
input_splitter
input_transformer_manager
input_transformers_cleanup
input_transformers_post
raw_input_original
set_next_input
show_rewritten_input


Проверим, что из себя представляет `raw_input_original`

In [21]:
type(instance.raw_input_original)

builtin_function_or_method

In [22]:
instance.raw_input_original.__name__

'input'

In [23]:
instance.raw_input_original.__module__

'builtins'

In [24]:
help(instance.raw_input_original)

Help on built-in function input in module builtins:

input(prompt=None, /)
    Read a string from standard input.  The trailing newline is stripped.
    
    The prompt string, if given, is printed to standard output without a
    trailing newline before reading input.
    
    If the user hits EOF (*nix: Ctrl-D, Windows: Ctrl-Z+Return), raise EOFError.
    On *nix systems, readline is used if available.



Да, это и есть исходный `input` из модуля `builtins`, который похоже не используется. Но вернемся к той функции `input`, которую предоставлет Jupyter.  

<a id="use_case"></a>
## Как мы можем использовать полученные знания?
Вариант использования, о котром я давно думал - это контекстный менеджер для передачи в `input` одного и того же ответа.

In [25]:
from ipykernel.ipkernel import IPythonKernel
from contextlib import contextmanager

@contextmanager
def repeat(answer: str):
    try:
        ipython_input = IPythonKernel._input_request
        
        def mock_input(*args, **kwargs):
            return answer
        
        IPythonKernel._input_request = mock_input
        yield
    
    finally:
        IPythonKernel._input_request = ipython_input

In [26]:
with repeat('yes'):
    answer_1 = input('Continue?')
    answer_2 = input('Continue?')
    answer_3 = input('Continue?')
    
print(answer_1)
print(answer_2)
print(answer_3)

yes
yes
yes


Проверим, все ли правильно восстановилось (напомню, что ранее мы уже подменили `input` так, чтобы он нас приветствовал 'Hello world!')

In [27]:
input('Try to say something: ')

Hello world!
Try to say something: Good!


'Good!'

Да, это работает. Проверим контекстный менеджер на функции, имитирующей какой-то процесс.

In [28]:
def process():
    print('Starting the process...')
    print('Working...')
    for _ in range(3):
        print('Paused.')
        answer = input('Continue? (Y)es|(N)o :')
        if answer == 'Y':
            print('Resume and continue the process.')
        else:
            print('Breaking the process.')
            break
    print('Done.')

Вот как этот процесс выполняется без контекстного менеджера.

In [30]:
process()

Starting the process...
Working...
Paused.
Hello world!
Continue? (Y)es|(N)o :Y
Resume and continue the process.
Paused.
Hello world!
Continue? (Y)es|(N)o :Y
Resume and continue the process.
Paused.
Hello world!
Continue? (Y)es|(N)o :Y
Resume and continue the process.
Done.


Мы три раза вручную вводили 'Y'. А теперь запустим процесс с подменой функции ввода.

In [31]:
with repeat('Y'):
    process()

Starting the process...
Working...
Paused.
Resume and continue the process.
Paused.
Resume and continue the process.
Paused.
Resume and continue the process.
Done.


Проверим, как работает `input` после выхода из контекстного менеджера

In [32]:
input('Check the state of input: ')

Hello world!
Check the state of input: Okey


'Okey'

Ну и вернем все, как было

In [36]:
IPythonKernel._input_request = ipython_input

In [37]:
input('The End. ')

The End. Bye!


'Bye!'